# Train a Convolutional Variational Autoencoder 

Using the Conv-VAE package to train a convolutional variational autoencoder. The parameters of the training are describe by an experiment run of type "conv_vae". 

The result of runing the code in this notebook is the model files that are stored in the experiment directory. As the model files 

__Note:__ The result of running this code is the creation of a "final model". The encoder portion of this model is the component that can be used for the encoder of the robot. Once the training had been done, the subdir of this experiment and the checkpoint we want to use will need to updated into the experiment run-specific file in the model_subdir and model_checkpoint fields.


In [1]:
import sys
sys.path.append("..")
from settings import Config
import pathlib
from pprint import pprint
import shutil

import torch

# adding the Julian-8897-Conv-VAE-PyTorch into the path
sys.path.append(Config()["conv_vae"]["code_dir"])

# Oh, this hack was fixing something, but for me it is the other way around
#temp = pathlib.PosixPath
#pathlib.PosixPath = pathlib.WindowsPath

from conv_vae import get_conv_vae_config, create_configured_vae_json, train,  latest_training_run

Loading pointer config file: /home/ssheikholeslami/.config/BerryPicker/mainsettings.yaml
Loading machine-specific config file: /home/ssheikholeslami/SaharaBerryPickerData/settings-sahara.yaml


In [3]:
# If it is set to true, no actual copying will be done
dry_run = False

# Specify and load the experiment
run = "vp_convvae_128"
exp = Config().get_experiment("visual_proprioception", run)
pprint(exp)

No system dependent experiment file
 /home/ssheikholeslami/SaharaBerryPickerData/experiments-Config/visual_proprioception/vp_convvae_128_sysdep.yaml,
 that is ok, proceeding.
Configuration for experiment: visual_proprioception/vp_convvae_128 successfully loaded
{'data_dir': PosixPath('/home/ssheikholeslami/SaharaBerryPickerData/experiment_data/visual_proprioception/vp_convvae_128'),
 'encoding_size': 128,
 'epochs': 1000,
 'exp_run_sys_indep_file': PosixPath('/lustre/fs1/home/ssheikholeslami/BerryPicker/src/experiment_configs/visual_proprioception/vp_convvae_128.yaml'),
 'group_name': 'visual_proprioception',
 'loss': 'MSE',
 'name': 'conv-vae-128',
 'output_size': 6,
 'proprioception_input_file': 'train_inputs.pt',
 'proprioception_mlp_model_file': 'proprioception_mlp.pth',
 'proprioception_target_file': 'train_targets.pt',
 'proprioception_test_input_file': 'test_inputs.pt',
 'proprioception_test_target_file': 'test_targets.pt',
 'proprioception_testing_task': 'proprio_regressor_vali

In [5]:
# update the vae_config based on parameters from the experiment
# vae_config["trainer"]["epochs"] = exp["epochs"] # was 100
# vae_config["trainer"]["save_period"] = exp["save_period"] # 5
# vae_config["arch"]["args"]["latent_dims"] = exp["latent_dims"]

# FIXME: among other things, the problem is that this is not being written back into the config file

# print(exp["latent_dims"])

### Create the training data

The training data is the set of pictures from a demonstration. The following data copies the specific pictures and puts them the training directory specific to the run.

In [4]:
def copy_images_to_training_dir(taskname, training_image_dir):
    """Copy all the images from a specific task into the training image dir."""
    task_dir = pathlib.Path(demos_dir, taskname)
    # _, task_dir = ui_choose_task(offer_task_creation=True)

    for demo in task_dir.iterdir():
        if not demo.is_dir(): continue
        for item in demo.iterdir():
            if item.suffix != ".jpg": continue
            name = f"{demo.name}_{item.stem}.jpg"
            destination = pathlib.Path(training_image_dir, name)
            print(f"copy {item} to \n{destination}")
            if not dry_run:
                shutil.copyfile(item, destination)

In [5]:
import os
demos_top = pathlib.Path(Config()["demos"]["directory"])
demos_dir = pathlib.Path(demos_top, "demos")

subdir_count = sum(1 for item in demos_dir.iterdir() if item.is_dir())
print(f"Number of demo directories: {subdir_count}")

# Deciding on the location of the training data
training_data_dir = pathlib.Path(exp["data_dir"], exp["training_data_dir"])
# training_data_dir = pathlib.Path(Config()["conv_vae"]["training_data_dir"])
training_image_dir = pathlib.Path(training_data_dir, "Images")
if not os.path.exists(training_image_dir):
    training_image_dir.mkdir(exist_ok = False, parents=True)
    print(f"Directory '{training_image_dir}' created.")
else:
    print(f"Directory '{training_image_dir}' already exists.")

print(f"Training data dir={training_image_dir}")

# Define a set of common image file extensions
image_extensions = {".jpg", ".jpeg", ".png", ".gif", ".bmp", ".tiff", ".webp"}
# Count the image files
image_count = sum(1 for item in training_image_dir.iterdir() if item.suffix.lower() in image_extensions and item.is_file())

print(f"Number of image files in training dir: {image_count}")

if image_count == 0:
    taskname = exp['training_task']
    copy_images_to_training_dir(
        taskname = taskname, training_image_dir=training_image_dir)
else:
    print("There are already images in training image dir {training_image_dir}. Do not repeat the copying.")


Number of demo directories: 7


KeyError: 'training_data_dir'

### Copy files to the training dataset
Create the training data by copying all the files from the specific data to the training directory.

FIXME: maybe do this for a collection of different demos

# Train a ConvVAE on the training data

__Important__ After the training finished, in order to use the resulting system, one need to edit the run file (eg: vae_01.yaml) and enter into it the subdirectory of the model (eg. "0108_203019") and the name of the checkpoint we want to use. These might also be better put into the machine specific directory.

```
model_subdir: '1222_150723'
model_checkpoint: 'checkpoint-epoch100.pth'
```

In [5]:
# Create the vae configuration, based on the experiment

### Sahara's note: Just making sure weare  using gpu ####################
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device.type == "cuda":
    print("Training on GPU")
else:
    print("Training on CPU")

##########################################################################
file = create_configured_vae_json(exp)
print(file)
vae_config = get_conv_vae_config(file)

# actually run the training
print(f'Running the trainer from scratch for {vae_config["trainer"]["epochs"]}')
trainer = train(vae_config)

Training on GPU
/lustre/fs1/home/ssheikholeslami/BerryPicker/src/encoding_conv_vae/conv-vae-config-default.json
{'name': 'VAE_Robot', 'n_gpu': 1, 'arch': {'type': 'VanillaVAE', 'args': {'in_channels': 3, 'latent_dims': 128, 'flow': False}}, 'data_loader': {'type': 'CelebDataLoader', 'args': {'data_dir': '/home/ssheikholeslami/SaharaBerryPickerData/experiment_data/sensorprocessing_conv_vae/proprio_128/vae-training-data', 'batch_size': 64, 'shuffle': True, 'validation_split': 0.2, 'num_workers': 2}}, 'optimizer': {'type': 'Adam', 'args': {'lr': 0.005, 'weight_decay': 0.0, 'amsgrad': True}}, 'loss': 'elbo_loss', 'metrics': [], 'lr_scheduler': {'type': 'StepLR', 'args': {'step_size': 50, 'gamma': 0.1}}, 'trainer': {'epochs': 300, 'save_dir': '/home/ssheikholeslami/SaharaBerryPickerData/experiment_data/sensorprocessing_conv_vae/proprio_128/models', 'save_period': 5, 'verbosity': 2, 'monitor': 'min val_loss', 'early_stop': 10, 'tensorboard': True}}
/home/ssheikholeslami/SaharaBerryPickerData

2025-03-05 19:16:53.346083: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-05 19:16:57.580825: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741220218.562276  490536 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741220218.916924  490536 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-05 19:17:00.849951: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [6]:
# These are the metrics recorded
# they are of utils/util.py / MetricTracker which has a pandas dataframe as data
print(trainer.train_metrics)
print(trainer.valid_metrics)
#
trainer.train_metrics._data
# trainer.valid_metrics._data

,total,counts,average
loss,34176.149536,23,1485.919545
